# Índice

1. [Introducción a XGBoost](#introducción-a-xgboost)

   1.1. [Principales características y ventajas](#principales-características-y-ventajas)
   
      1.1.0 [Diferencias entre XGBoost y otros algoritmos de ensemble](#diferencias-entre-xgboost-y-otros-algoritmos-de-ensemble)

   1.2. [Matemática detrás de XGBoost](#matemática-detrás-de-xgboost)

      1.2.0 [Ajuste de hiperparámetros](#ajuste-de-hiperparámetros)

2. [Implementación del modelo](#implementación-del-modelo)

   2.0 [Paquetes](#paquetes)
   2.1 [Modelo](#modelo)


# Introducción a XGBoost

XGBoost (eXtreme Gradient Boosting) es un algoritmo de machine learning creado por Tianqi Chen. Es una implementación optimizada del algoritmo de Gradient Boosting.

XGBoost es un algoritmo de ensamble que construye modelos de forma secuencial, donde cada nuevo modelo intenta corregir los errores de los modelos anteriores. Utiliza el principio de gradient boosting, que optimiza una función de pérdida mediante el descenso de gradiente.

## Principales características y ventajas

- **Alto rendimiento**: Suele ofrecer mejores resultados que otros algoritmos en una variedad de datasets.
  
- **Velocidad y eficiencia**: Optimizado para ser rápido y eficiente en el uso de recursos.
  
- **Regularización**: Incluye regularización L1 (Lasso) y L2 (Ridge) que ayuda a prevenir el sobreajuste.
  
- **Manejo de valores faltantes**: Puede manejar automáticamente valores faltantes en los datos.
  
- **Flexibilidad**: Soporta múltiples funciones de pérdida y permite la personalización.
  
- **Parallelismo**: Aprovecha el paralelismo en la construcción de árboles.

### Diferencias entre XGBoost y otros algoritmos de ensemble

- **Random Forest**: Construye árboles de forma independiente y luego promedia (bagging), mientras que XGBoost construye árboles de forma secuencial (boosting).
  
- **Gradient Boosting tradicional**: XGBoost es una implementación más eficiente y escalable, con regularización y manejo de missing values.

## Matemática detrás de XGBoost

XGBoost optimiza una función de pérdida regularizada. El objetivo en cada paso es:
```
Obj = Σ L(y_i, y_hat_i) + Σ Ω(f_k)
```
donde:
- `L` es la función de pérdida (ej: log loss para clasificación)
- `Ω` es el término de regularización que penaliza la complejidad del modelo
- `f_k` es el árbol en la iteración k
El algoritmo utiliza gradientes de segundo orden (Hessianos) para optimizar la función de pérdida, lo que acelera la convergencia.

### Ajuste de hiperparámetros

Los hiperparámetros más importantes en XGBoost son:

- `n_estimators`: Número de árboles.
  
- `max_depth`: Profundidad máxima de los árboles.
  
- `learning_rate`: Tasa de aprendizaje (shrinkage).
  
- `subsample`: Fracción de muestras usadas para entrenar cada árbol.
  
- `colsample_bytree`: Fracción de características usadas para cada árbol.
  
- `scale_pos_weight`: Controla el balance de clases.
  
- `reg_alpha` (alpha): Regularización L1.
  
- `reg_lambda` (lambda): Regularización L2.
Se pueden ajustar con técnicas como GridSearchCV o RandomizedSearchCV.

# Implementación del modelo

## Paquetes

In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

## Modelo

In [ ]:
# Entrenar XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=np.sum(y == 0) / np.sum(y == 1)  # Balancear clases
)
xgb_model.fit(X_train_selected, y_train)

# Predecir y evaluar
y_pred_xgb = xgb_model.predict(X_test_selected)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_selected)[:, 1]

print("XGBoost - Rendimiento:")
print(classification_report(y_test, y_pred_xgb))
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba_xgb):.4f}")

# Comparar curvas ROC
from sklearn.metrics import roc_curve

fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)

plt.figure(figsize=(10, 8))
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_score(y_test, y_pred_proba):.4f})')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {roc_auc_score(y_test, y_pred_proba_xgb):.4f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curvas ROC - Comparación de Modelos')
plt.legend()
plt.grid(True)
plt.show()

# Matriz de confusión para XGBoost
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Fraude', 'Fraude'],
            yticklabels=['No Fraude', 'Fraude'])
plt.title('Matriz de Confusión - XGBoost')
plt.ylabel('Verdaderos')
plt.xlabel('Predichos')
plt.show()

In [ ]:
# Importancia de características
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

# Visualizar
xgb.plot_importance(model)
plt.show()